<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://cdn2.hubspot.net/hubfs/438089/docs/training/dblearning-banner.png" alt="Databricks Learning" width="555" height="64">
</div>

&copy; 2018 Databricks, Inc. All rights reserved.<br/>

#![Spark Logo Tiny](https://kpistoropen.blob.core.windows.net/collateral/roadshow/logo_spark_tiny.png) Getting Started with Azure Storage and Azure Data Lake 

**Databricks Mount Points:**
- Connect to our Azure Storage Account - https://docs.azuredatabricks.net/spark/latest/data-sources/azure/azure-storage.html
- Connect to our Azure Data Lake - https://docs.azuredatabricks.net/spark/latest/data-sources/azure/azure-datalake.html

##![Spark Logo Tiny](https://kpistoropen.blob.core.windows.net/collateral/roadshow/logo_spark_tiny.png) Connect to our Azure Storage Account

Next, let's connect to the read-only Blob store you'll have access to for data needed in this course.  We can easily mount data in blob stores to Azure Databricks for fast and scalable data storage

*Note:* You will have to have a cluster running to execute this code

**IMPORTANT** This notebook must be run using Azure Databricks runtime 4.0 or better.

In [6]:
dbutils.fs.help()

dbutils.fs provides utilities for working with FileSystems. Most methods in
this package can take either a DBFS path (e.g., "/foo" or "dbfs:/foo"), or
another FileSystem URI.

For more info about a method, use dbutils.fs.help("methodName") .

In notebooks, you can also use the %fs shorthand to access DBFS. The %fs shorthand maps
straightforwardly onto dbutils calls. For example, "%fs head --maxBytes=10000 /file/path"
translates into "dbutils.fs.head("/file/path", maxBytes = 10000)".
 fsutils cp(from: String, to: String, recurse: boolean = false): boolean -> Copies a file or directory, possibly across FileSystems head(file: String, maxBytes: int = 65536): String -> Returns up to the first 'maxBytes' bytes of the given file as a String encoded in UTF-8 ls(dir: String): Seq -> Lists the contents of a directory mkdirs(dir: String): boolean -> Creates the given directory if it does not exist, also creating any necessary parent directories mv(from: String, to: String, recurse: boolean = false): boolean -> Moves a file or directory, possibly across FileSystems put(file: String, contents: String, overwrite: boolean = false): boolean -> Writes the given String out to a file, encoded in UTF-8 rm(dir: String, recurse: boolean = false): boolean -> Removes a file or directory mount mount(source: String, mountPoint: String, encryptionType: String = "", owner: String = null, extraConfigs: Map = Map.empty[String, String]): boolean -> Mounts the given source directory into DBFS at the given mount point mounts: Seq -> Displays information about what is mounted within DBFS refreshMounts: boolean -> Forces all machines in this cluster to refresh their mount cache, ensuring they receive the most recent information unmount(mountPoint: String): boolean -> Deletes a DBFS mount point

In [7]:
#In case you have run this training before, you can unmount in order to be able to re-mount
try:
  dbutils.fs.unmount("/mnt/databricks-workshop-datasets") # Use this to unmount as needed
except:
  print("{} already unmounted".format("/mnt/databricks-workshop-datasets"))

/mnt/databricks-workshop-datasets/ has been unmounted.
 Out[ 4 ]: True

In [8]:
# This only needs to be ran once, globably. Once we have mounted the storage account no need need to do it again (unless you unmount). 
# These credentials DO NOT have write access

STORAGE_ACCOUNT = "channelsapublicprodblob"
CONTAINER = "channelsa-datasets"
MOUNT_POINT = "/mnt/databricks-workshop-datasets"
SAS_KEY = "?sv=2018-03-28&ss=b&srt=sco&sp=rwlac&se=2022-07-07T02:01:53Z&st=2019-03-24T19:01:53Z&spr=https&sig=o6rvr92oZH4nzdn7r4gR%2Bxv%2Fj%2BkOgv5BhXIfbTJYM%2Bg%3D"

#Define strings to be passed to the mount function
source_str = "wasbs://{container}@{storage_acct}.blob.core.windows.net/".format(container=CONTAINER, storage_acct=STORAGE_ACCOUNT)
conf_key = "fs.azure.sas.{container}.{storage_acct}.blob.core.windows.net".format(container=CONTAINER, storage_acct=STORAGE_ACCOUNT)

#Run the mount function using template in the documentation
try:
  dbutils.fs.mount(
    source = source_str,
    mount_point = MOUNT_POINT,
    extra_configs = {conf_key: SAS_KEY}
  )
except Exception as e:
  print("ERROR: {} already mounted. Run previous cells to unmount first".format(MOUNT_POINT))
  
#If needed to unmount, use this:
#try:
#  dbutils.fs.unmount(MOUNT_POINT) # Use this to unmount as needed
#except:
#  print("{} already unmounted".format(MOUNT_POINT))

In [9]:
%fs ls /mnt/databricks-workshop-datasets/

path,name,size
dbfs:/mnt/databricks-workshop-datasets/Contoso-retail/,Contoso-retail/,0
dbfs:/mnt/databricks-workshop-datasets/Example1folder/,Example1folder/,0
dbfs:/mnt/databricks-workshop-datasets/Example2folder/,Example2folder/,0


In [10]:
%fs ls /mnt/databricks-workshop-datasets/Contoso-retail/

path,name,size
dbfs:/mnt/databricks-workshop-datasets/Contoso-retail/dev/,dev/,0
dbfs:/mnt/databricks-workshop-datasets/Contoso-retail/initech/,initech/,0
dbfs:/mnt/databricks-workshop-datasets/Contoso-retail/structured-streaming/,structured-streaming/,0


If, by any chance, you cannot write to the local FileStore, you can use this mounted blob below

In [12]:
STORAGE_ACCOUNT = "channelsapublicexercises"
CONTAINER = "exercise-container"
MOUNT_POINT = "/mnt/databricks-workshop-exercises"
SAS_KEY = "?sv=2018-03-28&ss=b&srt=sco&sp=rwdlac&se=2021-11-27T03:35:07Z&st=2019-03-24T19:35:07Z&spr=https&sig=w%2Fp9iG2FGDlgNT716Kt3ZFnWQuUGlaxz3Bu4yVAVEwo%3D"

#Define strings to be passed to the mount function
source_str = "wasbs://{container}@{storage_acct}.blob.core.windows.net/".format(container=CONTAINER, storage_acct=STORAGE_ACCOUNT)
conf_key = "fs.azure.sas.{container}.{storage_acct}.blob.core.windows.net".format(container=CONTAINER, storage_acct=STORAGE_ACCOUNT)

#Run the mount function using template in the documentation
try:
  dbutils.fs.mount(
    source = source_str,
    mount_point = MOUNT_POINT,
    extra_configs = {conf_key: SAS_KEY}
  )
except Exception as e:
  print("ERROR: {} already mounted. Run previous cells to unmount first".format(MOUNT_POINT))
  
#If needed to unmount, use this:
#try:
#  dbutils.fs.unmount(MOUNT_POINT) # Use this to unmount as needed
#except:
#  print("{} already unmounted".format(MOUNT_POINT))

## Next Step

[Reading Data]($../2-ETL/2-01 Reading Data)

&copy; 2018 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>